In [1]:
import pandas as pd
from googleapiclient.discovery import build
import scrapetube

In [7]:
api_key = 'api_key'
channel_id = 'UChlv4GSd7OQl3js-jkLOnFA'
youtube = build('youtube', 'v3', developerKey=api_key)

In [5]:
videos = scrapetube.get_channel(channel_id)

video_ids = []

for video in videos:
    video_ids.append(video['videoId'])
    
len(video_ids)

8435

In [13]:
video_infos = []

for i in range(200):
    start = i * 50 + 3000
    end = (i + 1) * 50 + 3000
    video_request = youtube.videos().list(
        part="snippet",
        id=','.join(video_ids[start:end]))

    video_response = video_request.execute()

    for item in video_response['items']:
        title = item['snippet']['title']
        if ('글로벌 이슈체크' in title) or ('글로벌 마켓브리핑' in title) or ('직장인 vlog' in title):
            continue
        video_infos.append([item['snippet']['title'], item['snippet']['publishedAt'], item['id']])
        
df = pd.DataFrame(video_infos, columns=['title', 'date', 'id'])
print(len(df))
df.head()

5302


,title,date,id
0,[심층 인터뷰] 지금이 금리 인상의 적절한 타이밍이다? f.하나금융투자 이미선 연구위원,2021-08-26T01:02:19Z,ySArcJ5eLJQ
1,[미국 시장 완전 정복] 지치지 않는 미국 시장.. 계속되는 사상 최고가 행진 f....,2021-08-26T00:57:14Z,A65c_3aCz5M
2,"[뉴스3] 골드만삭스 ""한은 8월·11월 금리 인상…코스피 3750 목표가 유지""",2021-08-26T00:47:21Z,lD6ojF3-Ty0
3,"금리 인상 예고한 한국은행.. 금통위 결과는?_오늘아침 라이브_이항영, 이미선, 염승환",2021-08-26T00:32:02Z,5K-JSduTjd0
4,"[글로벌 개장시황] 최고치 갱신하는 수익 맛집 미국 주식, 여전히 늦지 않았다. f...",2021-08-25T14:47:44Z,xs19GBQCbJQ


In [14]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df

,title,date,id
0,[심층 인터뷰] 지금이 금리 인상의 적절한 타이밍이다? f.하나금융투자 이미선 연구위원,2021-08-26 01:02:19+00:00,ySArcJ5eLJQ
1,[미국 시장 완전 정복] 지치지 않는 미국 시장.. 계속되는 사상 최고가 행진 f....,2021-08-26 00:57:14+00:00,A65c_3aCz5M
2,"[뉴스3] 골드만삭스 ""한은 8월·11월 금리 인상…코스피 3750 목표가 유지""",2021-08-26 00:47:21+00:00,lD6ojF3-Ty0
3,"금리 인상 예고한 한국은행.. 금통위 결과는?_오늘아침 라이브_이항영, 이미선, 염승환",2021-08-26 00:32:02+00:00,5K-JSduTjd0
4,"[글로벌 개장시황] 최고치 갱신하는 수익 맛집 미국 주식, 여전히 늦지 않았다. f...",2021-08-25 14:47:44+00:00,xs19GBQCbJQ
...,...,...,...
5297,"오늘 아침 Page2 / 19.01.28 / 권순우, 곽상준",2019-01-27 23:43:10+00:00,NEZJy8PmubM
5298,"[풀영상] 오늘아침 Page2 - 김일구, 곽상준 19.01.25",2019-01-24 23:40:49+00:00,FsXHEfdpNsY
5299,신과함께 기본소득편 - 이원재 랩2050 대표 / 완전체 삼프로 라이브,2019-01-24 14:44:57+00:00,YWLt1HSgZKA
5300,"오늘 아침 Page2 / 19.01.24 / 김지윤, 곽상준",2019-01-23 23:45:10+00:00,IZm-WcH44zY


In [15]:
df = df.set_index('date')

In [16]:
df_21 = df[(pd.DatetimeIndex(df.index).year == 2021) & (pd.DatetimeIndex(df.index).month < 6)]
df_20 = df[(pd.DatetimeIndex(df.index).year == 2020) & (pd.DatetimeIndex(df.index).month >= 6)]

In [17]:
comments_21 = []

for video_id in df_21['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
 
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_21.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df0 = pd.DataFrame(comments_21, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
print(len(df0))
df0.head()

77495


,id,comment,author,comment_date,num_likes
0,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31T15:11:22Z,1
1,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01T07:28:36Z,1
2,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01T05:44:35Z,1
3,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01T03:29:26Z,1
4,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01T03:18:53Z,2


In [18]:
df_21 = df_21.reset_index()
df_21.rename(columns={'date': 'video_date'}, inplace=True)
df_21.head()

,video_date,title,id
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI
1,2021-05-31 14:12:18+00:00,"너도나도 인공지능!! 종목 선택은 어떻게?_글로벌 라이브_장의성, 권영배, 장우석",9i1eEnWcleg
2,2021-05-31 11:09:40+00:00,[심층 인터뷰] 다가오는 미국 고용보고서 발표.. 방향성 가늠자 된다? f.교보증권...,ScZs6W_scw8
3,2021-05-31 10:50:34+00:00,[증시 셔터맨] 3200 포인트 넘긴 코스피.. 오늘은 컨택트의 날이었다? f.교보...,YduXYrx_pd8
4,2021-05-31 10:24:25+00:00,박병창 부장 스페셜.. 진심을 전하는 시장 전망!_퇴근길 page2_박병창,G_juOvaWMQ8


In [19]:
sam_21 = pd.merge(df_21, df0, on='id', how='outer')
sam_21.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31T15:11:22Z,1
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01T07:28:36Z,1
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01T05:44:35Z,1
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01T03:29:26Z,1
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01T03:18:53Z,2


In [20]:
sam_21.to_csv('sampro_21_1.csv', header=['video_date', 'title', 'id', 'comment', 'author', 'comment_date', 'num_likes'], index=None)
sam_21 = pd.read_csv('sampro_21_1.csv')
sam_21.head()              

,video_date,title,id,comment,author,comment_date,num_likes
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31T15:11:22Z,1
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01T07:28:36Z,1
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01T05:44:35Z,1
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01T03:29:26Z,1
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01T03:18:53Z,2


In [ ]:
comments_20 = []

for video_id in df_20['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
 
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_20.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

In [26]:
df1 = pd.DataFrame(comments_20, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
print(len(df1))
df1.head()

113423


,id,comment,author,comment_date,num_likes
0,pRQjRH9AVrI,진짜 이 분은 책 쓰실만 합니다. 분명 인격적으로도 완성된 분일 것이라는 확신이 듭니다.,달빛의향기,2022-06-29T09:23:15Z,0
1,pRQjRH9AVrI,찌 인 한 눈썹.이 보임.... 부장님 말씀 너무 쉽게 친절하게 설명해주셔서 정말 ...,고고,2022-06-21T20:49:32Z,0
2,pRQjRH9AVrI,명강의..쉽게 배울수 없는 고급정보,쪄리,2022-06-10T18:17:08Z,0
3,pRQjRH9AVrI,“와... 이 분들을 한 자리에...? 😲”\r<br>삼프로TV가 아니면 절대 한 ...,삼프로TV_경제의신과함께,2022-05-09T07:36:33Z,0
4,pRQjRH9AVrI,42.24에서 소름입니다...,머니준,2022-05-03T01:31:10Z,0


In [29]:
df_20 = df_20.reset_index()
df_20.rename(columns={'date': 'video_date'}, inplace=True)
df_20.head()

,video_date,title,id
0,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI
1,2020-12-31 00:00:10+00:00,새해를 맞아 다시 한 번 뭉쳤다! 경제 유튜버 3인에게 들어보는 2021 투자의 길...,tb93cz7-RuM
2,2020-12-30 23:00:33+00:00,염블리 염승환 부장의 2020 경제 이슈 총정리 [신과함께 연말특집 1탄],w-dJ-c6rHhk
3,2020-12-30 15:52:47+00:00,장우석 본부장이 말하는 2021년 미국주식 투자포인트 f. 장우석 본부장,Cdfteh5MlRY
4,2020-12-30 15:52:00+00:00,2021년 디스플레이 산업에서 주목해야할 포인트 f. 김철중 연구원,4cxe0mg1vo0


In [30]:
sam_20 = pd.merge(df_20, df1, on='id', how='outer')
sam_20.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,진짜 이 분은 책 쓰실만 합니다. 분명 인격적으로도 완성된 분일 것이라는 확신이 듭니다.,달빛의향기,2022-06-29T09:23:15Z,0
1,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,찌 인 한 눈썹.이 보임.... 부장님 말씀 너무 쉽게 친절하게 설명해주셔서 정말 ...,고고,2022-06-21T20:49:32Z,0
2,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,명강의..쉽게 배울수 없는 고급정보,쪄리,2022-06-10T18:17:08Z,0
3,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,“와... 이 분들을 한 자리에...? 😲”\r<br>삼프로TV가 아니면 절대 한 ...,삼프로TV_경제의신과함께,2022-05-09T07:36:33Z,0
4,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,42.24에서 소름입니다...,머니준,2022-05-03T01:31:10Z,0


In [31]:
sam_20.to_csv('sampro_20_2.csv', header=['video_date', 'title', 'id', 'comment', 'author', 'comment_date', 'num_likes'], index=None)
sam_20 = pd.read_csv('sampro_20_2.csv')
sam_20.head()    

,video_date,title,id,comment,author,comment_date,num_likes
0,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,진짜 이 분은 책 쓰실만 합니다. 분명 인격적으로도 완성된 분일 것이라는 확신이 듭니다.,달빛의향기,2022-06-29T09:23:15Z,0
1,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,찌 인 한 눈썹.이 보임.... 부장님 말씀 너무 쉽게 친절하게 설명해주셔서 정말 ...,고고,2022-06-21T20:49:32Z,0
2,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,명강의..쉽게 배울수 없는 고급정보,쪄리,2022-06-10T18:17:08Z,0
3,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,“와... 이 분들을 한 자리에...? 😲”\r<br>삼프로TV가 아니면 절대 한 ...,삼프로TV_경제의신과함께,2022-05-09T07:36:33Z,0
4,2020-12-31 23:00:35+00:00,박병창 부장이 말하는 돈을 부르는 매매의 타이밍! [신과함께 연말특집 2탄],pRQjRH9AVrI,42.24에서 소름입니다...,머니준,2022-05-03T01:31:10Z,0


In [32]:
sam_1y = pd.concat([sam_21, sam_20])
sam_1y.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31T15:11:22Z,1
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01T07:28:36Z,1
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01T05:44:35Z,1
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01T03:29:26Z,1
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01T03:18:53Z,2


In [33]:
sam_1y.isnull().sum()  

video_date      0
title           0
id              0
comment         0
author          1
comment_date    0
num_likes       0
dtype: int64

In [34]:
sam_1y['video_date'] = pd.to_datetime(sam_1y['video_date'], utc=True)
sam_1y['comment_date'] = pd.to_datetime(sam_1y['comment_date'], utc=True)

sam_1y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190918 entries, 0 to 113422
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   video_date    190918 non-null  datetime64[ns, UTC]
 1   title         190918 non-null  object             
 2   id            190918 non-null  object             
 3   comment       190918 non-null  object             
 4   author        190917 non-null  object             
 5   comment_date  190918 non-null  datetime64[ns, UTC]
 6   num_likes     190918 non-null  int64              
dtypes: datetime64[ns, UTC](2), int64(1), object(4)
memory usage: 11.7+ MB


In [35]:
sam_1y['date_diff'] = 0

for idx in sam_1y.index:
    v = sam_1y.loc[idx, 'video_date']
    c = sam_1y.loc[idx, 'comment_date']
    diff = c - v
    sam_1y.loc[idx, 'date_diff'] = diff

print(sam_1y.info())
sam_1y.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190918 entries, 0 to 113422
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   video_date    190918 non-null  datetime64[ns, UTC]
 1   title         190918 non-null  object             
 2   id            190918 non-null  object             
 3   comment       190918 non-null  object             
 4   author        190917 non-null  object             
 5   comment_date  190918 non-null  datetime64[ns, UTC]
 6   num_likes     190918 non-null  int64              
 7   date_diff     190918 non-null  object             
dtypes: datetime64[ns, UTC](2), int64(1), object(5)
memory usage: 17.1+ MB
None


,video_date,title,id,comment,author,comment_date,num_likes,date_diff
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31 15:11:22+00:00,1,-1 days +23:59:02
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01 07:28:36+00:00,1,0 days 16:16:16
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01 05:44:35+00:00,1,0 days 14:32:15
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01 03:29:26+00:00,1,0 days 12:17:06
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01 03:18:53+00:00,2,0 days 12:06:33


In [37]:
import datetime

In [38]:
sam_1y['market_hour'] = False
days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']

for idx in sam_1y.index:
    h = sam_1y.iloc[idx, 5].hour
    m = sam_1y.iloc[idx, 5].minute
    w = days[sam_1y.iloc[idx, 5].weekday()]
    
    if w != '토요일' and w != '일요일':
        if (datetime.time(h, m) > datetime.time(9, 0)) and (datetime.time(h, m) < datetime.time(15, 30)):
            sam_1y.loc[idx, 'market_hour'] = True
            
sam_1y.head()

,video_date,title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31 15:11:22+00:00,1,-1 days +23:59:02,True
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01 07:28:36+00:00,1,0 days 16:16:16,False
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01 05:44:35+00:00,1,0 days 14:32:15,False
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01 03:29:26+00:00,1,0 days 12:17:06,False
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01 03:18:53+00:00,2,0 days 12:06:33,False


In [39]:
sam_1y.to_csv('sampro_1y_2.csv', index=False)

In [40]:
sam_1y_1 = pd.read_csv('sampro_1y.csv')
sam_1y_2 = pd.read_csv('sampro_1y_2.csv')

In [41]:
sam_1y_1

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07,True
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54,False
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55,False
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56,False
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22,False
...,...,...,...,...,...,...,...,...,...
274493,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,블랙핑크와 bts는 전세계 1위입니다,서프라이즈슈퍼,2021-06-01 00:53:22+00:00,0,0 days 00:09:20,False
274494,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,잘나다니까. 그저 앵커나 모든 사람이 잘해서 잘나갈 수 밖에 없다고 알지요. 그것은...,조금성,2021-06-01 00:51:03+00:00,0,0 days 00:07:01,False
274495,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,오늘도 좋은 내용 감사감사~,오리,2021-06-01 00:50:35+00:00,2,0 days 00:06:33,False
274496,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,스팩관련주 너무 도박인가여?,Gu Park,2021-06-01 00:43:28+00:00,3,-1 days +23:59:26,False


In [42]:
sam_1y_2

,video_date,title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31 15:11:22+00:00,1,-1 days +23:59:02,True
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01 07:28:36+00:00,1,0 days 16:16:16,False
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01 05:44:35+00:00,1,0 days 14:32:15,False
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01 03:29:26+00:00,1,0 days 12:17:06,False
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01 03:18:53+00:00,2,0 days 12:06:33,False
...,...,...,...,...,...,...,...,...,...
190913,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,타켓이 아니라 타겟이라고 해요 ㅎㅎㅎ,Joshua Kwak,2020-06-01 00:16:31+00:00,1,0 days 00:15:02,False
190914,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,은행주 갈때됐지...,pizza0722,2020-06-01 00:13:28+00:00,2,0 days 00:11:59,False
190915,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,미국 초청받은건 가면됩니다...ㅎㅎ<br>한국을 중립국으로 가만히 뒀으면 좋겠다,volvo,2020-06-01 00:06:57+00:00,4,0 days 00:05:28,False
190916,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,은행주 오른다~,Outputor,2020-06-01 00:06:33+00:00,2,0 days 00:05:04,False


In [46]:
sam_1y_2.rename(columns={'title': 'video_title'}, inplace=True)

In [47]:
sam_1y_2.to_csv('sampro_1y_2.csv')
sam_1y_2.head()

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,삼프로TV 주식대학 &#39;봄학기&#39;\r<br>👉 염블리의 친절한 주식 기초...,삼프로TV_경제의신과함께,2021-05-31 15:11:22+00:00,1,-1 days +23:59:02,True
1,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,"소상공인 입장에서 페이스북, 인스타그램 소액 광고는 정말 좋은 것 같아요!",humming cake,2021-06-01 07:28:36+00:00,1,0 days 16:16:16,False
2,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,언능 쾌유를 빕니다..,이준경,2021-06-01 05:44:35+00:00,1,0 days 14:32:15,False
3,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,엔베디아 비중 젤 큰 Etf는 물렷는데 흑흑,Breath of Nature,2021-06-01 03:29:26+00:00,1,0 days 12:17:06,False
4,2021-05-31 15:12:20+00:00,[글로벌 개장시황] 'Sell In May'는 아니었지만 체감상 안좋은 이유는? f...,Nnx3FGfaukI,역시나 오늘도 즐거운 특강 감사드려요. 장엔젤님. 엔비디아 페이스북 설명 감사합니다...,비트크리스탈,2021-06-01 03:18:53+00:00,2,0 days 12:06:33,False


In [48]:
sam_2y = pd.concat([sam_1y_1, sam_1y_2])
sam_2y

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07,True
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54,False
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55,False
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56,False
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22,False
...,...,...,...,...,...,...,...,...,...
190913,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,타켓이 아니라 타겟이라고 해요 ㅎㅎㅎ,Joshua Kwak,2020-06-01 00:16:31+00:00,1,0 days 00:15:02,False
190914,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,은행주 갈때됐지...,pizza0722,2020-06-01 00:13:28+00:00,2,0 days 00:11:59,False
190915,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,미국 초청받은건 가면됩니다...ㅎㅎ<br>한국을 중립국으로 가만히 뒀으면 좋겠다,volvo,2020-06-01 00:06:57+00:00,4,0 days 00:05:28,False
190916,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,은행주 오른다~,Outputor,2020-06-01 00:06:33+00:00,2,0 days 00:05:04,False


In [49]:
sam_2y.to_csv('sampro_2y.csv', index=False)

In [51]:
sam_2y = pd.read_csv('sampro_2y.csv')

In [53]:
sam_2y

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07,True
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54,False
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55,False
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56,False
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22,False
...,...,...,...,...,...,...,...,...,...
465411,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,타켓이 아니라 타겟이라고 해요 ㅎㅎㅎ,Joshua Kwak,2020-06-01 00:16:31+00:00,1,0 days 00:15:02,False
465412,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,은행주 갈때됐지...,pizza0722,2020-06-01 00:13:28+00:00,2,0 days 00:11:59,False
465413,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,미국 초청받은건 가면됩니다...ㅎㅎ<br>한국을 중립국으로 가만히 뒀으면 좋겠다,volvo,2020-06-01 00:06:57+00:00,4,0 days 00:05:28,False
465414,2020-06-01 00:01:29+00:00,"은행주 사는 개인들, 또 '버핏 반대로 하기' 괜찮을까?_오늘아침 page2_권순우...",SqpiyYAZYR4,은행주 오른다~,Outputor,2020-06-01 00:06:33+00:00,2,0 days 00:05:04,False


In [52]:
len(sam_2y)

465416